In [4]:
#Really need these
import pandas as pd 
import numpy as np
from numpy import *


#Handy for debugging
import gc
import time
import warnings
import os

#Date stuff
from datetime import datetime
from datetime import timedelta

#Do some statistics
from scipy.misc import imread
from scipy import sparse
import scipy.stats as ss
import math

#Nice graphing tools
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns


#Machine learning tools
from sklearn.ensemble import RandomForestRegressor
from sklearn.utils.validation import check_X_y, check_is_fitted
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix
from scipy import sparse

## Keras for deep learning
import keras
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.layers import Bidirectional
from keras.models import Sequential
from keras import regularizers
from keras import optimizers

## Performance measures
from sklearn.metrics import mean_squared_error

Using TensorFlow backend.


In [7]:
print('Reading data...')
print(os.listdir('data/'))
print(os.listdir('data/train'))

Reading data...
['train', 'test', 'sample_submission']
['train.csv', 'item_data.csv', 'view_log.csv']


In [13]:
train=pd.read_csv('data/train/train.csv')
test=pd.read_csv('data/test/test.csv')

In [14]:
train.head()

,impression_id,impression_time,user_id,app_code,os_version,is_4G,is_click
0,c4ca4238a0b923820dcc509a6f75849b,2018-11-15 00:00:00,87862,422,old,0,0
1,45c48cce2e2d7fbdea1afc51c7c6ad26,2018-11-15 00:01:00,63410,467,latest,1,1
2,70efdf2ec9b086079795c442636b55fb,2018-11-15 00:02:00,71748,259,intermediate,1,0
3,8e296a067a37563370ded05f5a3bf3ec,2018-11-15 00:02:00,69209,244,latest,1,0
4,182be0c5cdcd5072bb1864cdee4d3d6e,2018-11-15 00:02:00,62873,473,latest,0,0


In [15]:
test.head()

,impression_id,impression_time,user_id,app_code,os_version,is_4G
0,a9e7126a585a69a32bc7414e9d0c0ada,2018-12-13 07:44:00,44754,127,latest,1
1,caac14a5bf2ba283db7708bb34855760,2018-12-13 07:45:00,29656,44,latest,0
2,13f10ba306a19ce7bec2f3cae507b698,2018-12-13 07:46:00,25234,296,latest,1
3,39c4b4dc0e9701b55a0a4f072008fb3f,2018-12-13 07:47:00,22988,207,latest,1
4,bf5a572cca75f5fc67f4b14e58b11d70,2018-12-13 07:48:00,35431,242,latest,1


In [16]:
train['datetime'] = pd.to_datetime(train['impression_time'])

test['datetime'] = pd.to_datetime(test['impression_time'])


train.head()

,impression_id,impression_time,user_id,app_code,os_version,is_4G,is_click,datetime
0,c4ca4238a0b923820dcc509a6f75849b,2018-11-15 00:00:00,87862,422,old,0,0,2018-11-15 00:00:00
1,45c48cce2e2d7fbdea1afc51c7c6ad26,2018-11-15 00:01:00,63410,467,latest,1,1,2018-11-15 00:01:00
2,70efdf2ec9b086079795c442636b55fb,2018-11-15 00:02:00,71748,259,intermediate,1,0,2018-11-15 00:02:00
3,8e296a067a37563370ded05f5a3bf3ec,2018-11-15 00:02:00,69209,244,latest,1,0,2018-11-15 00:02:00
4,182be0c5cdcd5072bb1864cdee4d3d6e,2018-11-15 00:02:00,62873,473,latest,0,0,2018-11-15 00:02:00


In [17]:
train['day'] = train['datetime'].dt.weekday
train['hour'] = train['datetime'].dt.hour

test['day'] = test['datetime'].dt.weekday
test['hour'] = test['datetime'].dt.hour

train['hour1'] = train['hour']%4
test['hour1'] = test['hour']%4


train.head()

,impression_id,impression_time,user_id,app_code,os_version,is_4G,is_click,datetime,day,hour,hour1
0,c4ca4238a0b923820dcc509a6f75849b,2018-11-15 00:00:00,87862,422,old,0,0,2018-11-15 00:00:00,3,0,0
1,45c48cce2e2d7fbdea1afc51c7c6ad26,2018-11-15 00:01:00,63410,467,latest,1,1,2018-11-15 00:01:00,3,0,0
2,70efdf2ec9b086079795c442636b55fb,2018-11-15 00:02:00,71748,259,intermediate,1,0,2018-11-15 00:02:00,3,0,0
3,8e296a067a37563370ded05f5a3bf3ec,2018-11-15 00:02:00,69209,244,latest,1,0,2018-11-15 00:02:00,3,0,0
4,182be0c5cdcd5072bb1864cdee4d3d6e,2018-11-15 00:02:00,62873,473,latest,0,0,2018-11-15 00:02:00,3,0,0


In [18]:
train = train.drop(columns=['impression_id', 'impression_time','datetime', 'hour',])
test = test.drop(columns=['impression_time','datetime', 'hour', ])
train.head()

,user_id,app_code,os_version,is_4G,is_click,day,hour1
0,87862,422,old,0,0,3,0
1,63410,467,latest,1,1,3,0
2,71748,259,intermediate,1,0,3,0
3,69209,244,latest,1,0,3,0
4,62873,473,latest,0,0,3,0


In [19]:
print(train.isnull().sum(axis=0))
print(test.isnull().sum(axis = 0))

user_id       0
app_code      0
os_version    0
is_4G         0
is_click      0
day           0
hour1         0
dtype: int64
impression_id    0
user_id          0
app_code         0
os_version       0
is_4G            0
day              0
hour1            0
dtype: int64


In [21]:
train.describe()

,user_id,app_code,is_4G,is_click,day,hour1
count,237609.000000,237609.000000,237609.000000,237609.000000,237609.000000,237609.000000
mean,46454.526828,249.099971,0.361312,0.045714,2.852977,1.418141
std,26802.726666,135.213609,0.480382,0.208864,2.007192,1.124943
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,23197.000000,163.000000,0.000000,0.000000,1.000000,0.000000
50%,46597.000000,213.000000,0.000000,0.000000,3.000000,1.000000
75%,69684.000000,385.000000,1.000000,0.000000,5.000000,2.000000
max,92586.000000,522.000000,1.000000,1.000000,6.000000,3.000000


In [22]:
test.describe()

,user_id,app_code,is_4G,day,hour1
count,90675.00000,90675.000000,90675.000000,90675.000000,90675.000000
mean,46417.71013,254.793703,0.357872,3.089032,1.424097
std,26835.33356,133.411434,0.479377,2.061584,1.131886
min,1.00000,0.000000,0.000000,0.000000,0.000000
25%,23029.00000,190.000000,0.000000,1.000000,0.000000
50%,46557.00000,213.000000,0.000000,3.000000,1.000000
75%,69764.50000,386.000000,1.000000,5.000000,2.000000
max,92586.00000,522.000000,1.000000,6.000000,3.000000


In [23]:
from sklearn.preprocessing import LabelEncoder
train = train.astype('object')
test = test.astype('object')
col_with_null = ['user_id', 'os_version']
cols = ['app_code', 'is_4G']
for col in col_with_null:
    lbl = LabelEncoder()
    temp_train = train[col]
    temp_test = test[col]
    mask = train[col].isnull()
    mask1 = test[col].isnull()
    train[col] = train[col].astype(str)
    test[col] = test[col].astype(str)
    lbl.fit(list(train[col].values) + list(test[col].values))
    train[col] = lbl.transform(list(train[col].values))
    test[col] = lbl.transform(list(test[col].values))
    train[col] = train[col].where(~mask, temp_train)
    test[col] = test[col].where(~mask1, temp_test)
    
for col in cols:
    lbl = LabelEncoder()
    lbl.fit(list(train[col].values) + list(test[col].values))
    train[col] = lbl.transform(list(train[col].values))
    test[col] = lbl.transform(list(test[col].values))print(confusion_matrix(y_test, np.round(lrc_pred)))
print(roc_auc_score(y_test, np.round(lrc_pred)))
print(auc(y_test,  np.round(lrc_pred)))

In [24]:
train = train.astype(int)
for col in test.columns[1:]: test[col] = test[col].astype(int)

In [26]:
from sklearn.model_selection import train_test_split

y = train['is_click'].astype('float')
X = train.drop(columns=['is_click'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [27]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=50, random_state=23)
model = clf.fit(X_train, y_train)

In [28]:
y_pred = clf.predict_proba(X_test)[:,1]

In [31]:
print(accuracy_score(y_test,np.round(y_pred)))

0.9408274062539456


In [32]:
from sklearn.metrics import confusion_matrix, roc_auc_score, accuracy_score as auc 

In [33]:
print(confusion_matrix(y_test, np.round(y_pred)))
print(roc_auc_score(y_test, np.round(y_pred)))
print(auc(y_test, np.round(y_pred)))

[[44601   783]
 [ 2029   109]]
0.5168647250354814
0.9408274062539456


In [34]:
lrc = LogisticRegression()

In [35]:
lrc_model = lrc.fit(X_train, y_train)

/home/citigo/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [39]:
lrc_pred = lrc.predict_proba(X_test)[:,1]

In [42]:
print(confusion_matrix(y_test, np.round(lrc_pred)))
print(roc_auc_score(y_test, np.round(lrc_pred)))
print(auc(y_test,  np.round(lrc_pred)))

[[45384     0]
 [ 2138     0]]
0.5
0.9550103110138463


In [43]:
from sklearn.svm import LinearSVC
lsvc = LinearSVC()
lsvc_model = lsvc.fit(X_train,y_train)

/home/citigo/.local/lib/python3.6/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [44]:
lsvc_pred = lsvc.predict(X_test)

In [45]:
print(confusion_matrix(y_test, np.round(lsvc_pred)))
print(roc_auc_score(y_test, np.round(lsvc_pred)))
print(auc(y_test,  np.round(lsvc_pred)))

[[45384     0]
 [ 2138     0]]
0.5
0.9550103110138463


In [56]:
clfs = Sequential([
    Dense(units=16, kernel_initializer='uniform', input_dim=30, activation='relu'),
    Dense(units=18, kernel_initializer='uniform', activation='relu'),
    Dropout(0.25),
    Dense(20, kernel_initializer='uniform', activation='relu'),
    Dense(24, kernel_initializer='uniform', activation='relu'),
    Dense(1, kernel_initializer='uniform', activation='sigmoid')
])

W0827 16:46:49.856403 140697428162368 deprecation.py:506] From /home/citigo/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:2888: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [58]:
clfs.summary()dnn with keras

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 16)                496       
_________________________________________________________________
dense_13 (Dense)             (None, 18)                306       
_________________________________________________________________
dropout_1 (Dropout)          (None, 18)                0         
_________________________________________________________________
dense_14 (Dense)             (None, 20)                380       
_________________________________________________________________
dense_15 (Dense)             (None, 24)                504       
_________________________________________________________________
dense_16 (Dense)             (None, 1)                 25        
Total params: 1,711
Trainable params: 1,711
Non-trainable params: 0
_________________________________________________________________


In [59]:
clfs.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

W0827 16:48:21.668704 140697428162368 deprecation.py:323] From /home/citigo/.local/lib/python3.6/site-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [64]:
train.shape

(237609, 7)

In [65]:
model = Sequential()
BatchNormalization()
model.add(Dense(237500,input_dim=7,activation='relu'))
BatchNormalization()
Dropout(0.5)
model.add(Dense(100,input_dim=237500,activation='relu'))
BatchNormalization()
Dropout(0.8)
model.add(Dense(50))
BatchNormalization()
model.add(Dense(1))
#sgd = SGD(lr=0.01)
model.compile( optimizer='adam',loss='mse',metrics=['mean_squared_error'])

In [70]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),nb_epoch=30,batch_size=100,verbose=2)

ValueError: Error when checking input: expected dense_17_input to have shape (None, 7) but got array with shape (190087, 6)